<a href="https://colab.research.google.com/github/Sogo95/Resume-CNN-Daily/blob/main/summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Objectif :
Mettre en pratique les concepts de modèle encoder-decoder en construisant un modèle capable de générer un résumé automatique d'un texte.


1.	Téléchargez et préparez un jeu de données : Utilisez un jeu de données contenant des textes longs ainsi que leurs résumés. Par exemple, vous pouvez utiliser le jeu de données CNN/Daily Mail disponible sur Hugging Face Datasets. Préparez les données en suivant ces étapes :
•	Charger les textes et les résumés associés.
•	Effectuer un prétraitement (nettoyage du texte, suppression des caractères spéciaux, etc.).
•	Diviser les données en ensembles d’entraînement, de validation et de test.


In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Charger les textes et les résumés associés.

In [2]:
import os
from datasets import load_dataset

# Charger le jeu de données CNN/Daily Mail
ds = load_dataset("abisee/cnn_dailymail", '3.0.0')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [3]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})


In [4]:
ds.shape

{'train': (287113, 3), 'validation': (13368, 3), 'test': (11490, 3)}

In [5]:
print(ds['train'][0]['article'])  # Texte de l'article
print(ds['train'][0]['highlights'])  # Résumé (highlights)

LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details of how

Effectuer un prétraitement (nettoyage du texte, suppression des caractères spéciaux, etc.

In [ ]:
import re
from nltk.stem import WordNetLemmatizer
import nltk
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Télécharger les ressources pour la lemmatisation
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')

# Initialiser le lemmatizer
lemmatizer = WordNetLemmatizer()

# Fonction de nettoyage et lemmatisation
def netoy_lemmat(text):
    # Supprimer les balises HTML
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'\s+', ' ', text)  # Supprimer les espaces multiples
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Supprimer les caractères spéciaux
    # 4. Suppression des chiffres (optionnelle)
    text = re.sub(r'\d+', '', text)
    tokens = nltk.word_tokenize(text)  # Tokenisation
    lemmatized = [lemmatizer.lemmatize(token.lower()) for token in tokens]  # Lemmatisation
    return ' '.join(lemmatized)



# Appliquer le nettoyage et la lemmatisation
ds = ds.map(lambda x: {'article': netoy_lemmat(x['article']),
                        'highlights': netoy_lemmat(x['highlights'])})

print("Traitement  terminés.")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Diviser les données en ensembles d’entraînement, de validation et de test.


In [ ]:
# Division en 80/10/10
train_testvalid = ds["train"].train_test_split(test_size=0.2, seed=42)
test_valid = train_testvalid["test"].train_test_split(test_size=0.5, seed=42)

# Fusion des ensembles pour obtenir entraînement, validation, et test
train_data = train_testvalid["train"]
val_data = test_valid["train"]
test_data = test_valid["test"]

print(f"Entraînement : {len(train_data)}, Validation : {len(val_data)}, Test : {len(test_data)}")

Modèle de résumé extractif (Utiliser le jeu de données de tests uniquement)

Approche :

Utiliser des méthodes de résumé extractif en considerant un modele de type encodeur comme BERT, …

Implémentation :

Extraire les phrases clés du texte en utilisant les techniques de votre choix.

Évaluation :

Évaluer la qualité des résumés extraits par rapport aux résumés de référence en utilisant les métriques BLEU et ROUGE.



In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Charger le tokenizer et le modèle BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

# Move model and data to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)




In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import nltk


# Téléchargement de punkt pour la tokenisation de phrases
nltk.download("punkt")

# Function to extract sentence embeddings
def get_sentence_embeddings(text):
    sentences = nltk.sent_tokenize(text)  # Split text into sentences
    embeddings = []

    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=512)

        # Move inputs to the same device as the model
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            sentence_embedding = outputs.last_hidden_state.mean(dim=1).squeeze()
            embeddings.append(sentence_embedding.cpu().numpy()) # Move embeddings back to CPU to avoid similar issues later

    return sentences, embeddings




In [ ]:
import numpy as np

# Fonction pour extraire les phrases clés
def extract_key_phrases(sentences, embeddings, num_sentences=3):
    # Calculer l'embedding moyen pour le document complet
    doc_embedding = np.mean(embeddings, axis=0)
    similarities = cosine_similarity([doc_embedding], embeddings).flatten()

    # Trier les phrases par similarité décroissante
    top_indices = similarities.argsort()[-num_sentences:][::-1]
    key_phrases = [sentences[i] for i in top_indices]

    return key_phrases

for i in range(10): # Extraction des phrases clés pour les 10 premièrs articles
    example = test_data[i]
    text = example["article"]  # Assurez-vous que "article" est la clé correcte pour accéder au texte dans votre test_data
    sentences, embeddings = get_sentence_embeddings(text)
    key_phrases = extract_key_phrases(sentences, embeddings, num_sentences=3)

    print("Article original :", text)
    print("Phrases clés extraites :")
    for phrase in key_phrases:
        print("-", phrase)
    print("\n")

## Évaluation :

Évaluer la qualité des résumés extraits par rapport aux résumés de référence en utilisant les métriques BLEU et ROUGE.



In [ ]:
!pip install rouge_score

In [ ]:
!pip install evaluate

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

def evaluate_summary(reference_summary, extracted_summary):
    # Calcul BLEU
    reference = [reference_summary.split()]
    candidate = extracted_summary[0].split()
    # Use SmoothingFunction
    smoothing_function = SmoothingFunction().method4
    bleu_score = sentence_bleu(reference, candidate, smoothing_function=smoothing_function)
    #bleu_score = sentence_bleu(reference, candidate)

    # Calcul ROUGE
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    rouge_scores = scorer.score(reference_summary, extracted_summary[0]) # Use extracted_summary[0]

    return bleu_score, rouge_scores


# Initialisation des scores cumulés
total_bleu_score = 0
total_rouge_scores = {"rouge1": 0, "rouge2": 0, "rougeL": 0}
num_documents = len(test_data)

# Évaluation pour tous les documents de test dans test_data
for i in range(num_documents):
    sample_text = test_data[i]["article"]  # Article
    reference_summary = test_data[i]["highlights"]  # Résumé de référence

    sentences, embeddings = get_sentence_embeddings(sample_text)
    extracted_summary = extract_key_phrases(sentences, embeddings)

    bleu, rouge = evaluate_summary(reference_summary, extracted_summary)

    # Cumul des scores
    total_bleu_score += bleu
    total_rouge_scores["rouge1"] += rouge["rouge1"].fmeasure
    total_rouge_scores["rouge2"] += rouge["rouge2"].fmeasure
    total_rouge_scores["rougeL"] += rouge["rougeL"].fmeasure

# Calcul des moyennes
average_bleu_score = total_bleu_score / num_documents
average_rouge_scores = {key: score / num_documents for key, score in total_rouge_scores.items()}

# Affichage des résultats globaux
print("Score moyen BLEU :", average_bleu_score)
print("Scores moyens ROUGE :", average_rouge_scores)


  # Construction du modèle Encoder-Decoder
   Utilisez une architecture Transformer basée sur un modèle pré-entraîné tel que BART ou T5. Ces modèles sont particulièrement adaptés pour les tâches de résumé de texte.

    L'encoder reçoit le texte original et transforme les séquences d'entrée en une représentation cachée.
    Le decoder génère un résumé à partir de cette représentation cachée.

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer, AdamW
import torch

# Chargement du modèle et du tokenizer
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

# Définir l'optimiseur
optimizer = AdamW(model.parameters(), lr=5e-5)

# Définir l'appareil (GPU si disponible, sinon CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Fonction de prétraitement
def preprocess_function(examples):
    # Tokenisation des articles et des résumés
    inputs = tokenizer(examples["article"], max_length=1024, padding="max_length", truncation=True)
    labels = tokenizer(examples["highlights"], max_length=150, padding="max_length", truncation=True)

    # Ensure labels are a list of lists with the correct dimensions
    labels["input_ids"] = [[label for label in input_id] for input_id in labels["input_ids"]]

    # Retourner les entrées et les labels tokenisés
    inputs["labels"] = labels["input_ids"]
    return inputs

# Appliquer le prétraitement aux jeux de données
train_data = train_data.map(preprocess_function, batched=True)
val_data = val_data.map(preprocess_function, batched=True)
test_data = test_data.map(preprocess_function, batched=True)

# Boucle d'entraînement
num_epochs = 3
for epoch in range(num_epochs):
    model.train()  # Passer en mode entraînement
    for batch in train_data:
        # Charger les entrées et les labels sur le device
        inputs = {
            "input_ids": torch.tensor(batch["input_ids"]).to(device),
            "attention_mask": torch.tensor(batch["attention_mask"]).to(device),
            "labels": torch.tensor(batch["labels"]).to(device)
        }

        # Calculer la perte
        outputs = model(**inputs)
        loss = outputs.loss

        # Backpropagation et mise à jour des poids
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Validation
    model.eval()  # Passer en mode évaluation
    with torch.no_grad():  # Pas de gradient pendant la validation
        val_loss = 0
        for batch in val_data:
            inputs = {
                "input_ids": torch.tensor(batch["input_ids"]).to(device),
                "attention_mask": torch.tensor(batch["attention_mask"]).to(device),
                "labels": torch.tensor(batch["labels"]).to(device)
            }
            outputs = model(**inputs)
            val_loss += outputs.loss.item()  # Ajouter la perte de ce batch

        print(f"Epoch {epoch+1}: Validation Loss: {val_loss / len(val_data)}")



4. Évaluation des performances

Génération de résumés :

Générer des résumés à partir du jeu de test pour l’approche abstraite.

Comparaison des approches :

Évaluer les résumés générés (abstrait et extractif) en utilisant les métriques ROUGE et BLEU.
Comparer les scores et analyser les résultats.


In [ ]:
# Fonction pour générer un résumé abstrait
def generate_summary_abstractive(article):
    inputs = tokenizer_abstractive(article, return_tensors="pt", max_length=1024, truncation=True, padding="max_length").to(device)

    # Générer le résumé
    summary_ids = model.generate(inputs["input_ids"], max_length=150, num_beams=4, length_penalty=2.0, early_stopping=True)

    # Décoder le résumé généré
    summary = tokenizer_abstractive.decode(summary_ids[0], skip_special_tokens=True)

    return summary

# Liste pour stocker les résumés générés
generated_abstractive_summaries = []

# Générer des résumés pour chaque article dans le jeu de test
for batch in tqdm(test_data):
    # Décoder l'article à partir des IDs
    article = tokenizer_abstractive.decode(batch["input_ids"][0], skip_special_tokens=True)

    # Générer le résumé pour cet article
    generated_abstractive = generate_summary_abstractive(article)

    # Ajouter le résumé généré à la liste
    generated_abstractive_summaries.append(generated_abstractive)

# Afficher les premiers résumés générés pour vérifier
for i in range(10):  # Afficher les 3 premiers résumés générés
    print(f"Article {i+1}:")
    print(f"Résumé généré : {generated_abstractive_summaries[i]}")
    print("\n")

 Calcul des métriques ROUGE ou BLEU

In [ ]:


# Importer la bibliothèque evaluate
import evaluate

# Charger la métrique ROUGE
rouge_metric = evaluate.load("rouge")

# Comparaison des résumés générés avec les résumés de référence
references = [sample['article'] for sample in test_sample]
rouge_scores = rouge_metric.compute(predictions=summaries, references=references)

# Affichage des résultats ROUGE
print("Scores ROUGE:", rouge_scores)



In [ ]:

# Charger la métrique BLEU
bleu_metric = evaluate.load("bleu")

# Pour BLEU, les résumés de référence doivent être sous forme de liste de listes de mots
references_bleu = [[ref.split()] for ref in references]  # Chaque référence doit être une liste de listes de mots
predictions_bleu = [pred.split() for pred in summaries]  # Les prédictions doivent être sous forme de listes de mots

# Calcul des scores BLEU
bleu_score = bleu_metric.compute(predictions=summaries, references=references_bleu)

# Affichage du score BLEU
print("Score BLEU:", bleu_score)
